In [1]:
import yaml
import numpy as np
import cv2
from imutils.object_detection import non_max_suppression

def perform_classification(video_src, cascade_src):
    cap = cv2.VideoCapture(video_src)
    car_cascade = cv2.CascadeClassifier(cascade_src)
    while True:
        ret, img = cap.read()
        if (type(img) == type(None)):
            print('Video-ul nu este gasit')
            break
        image_scaled = cv2.resize(img, None, fx=0.6, fy=0.6)
        gray = cv2.cvtColor(image_scaled, cv2.COLOR_BGR2GRAY)
        cars = car_cascade.detectMultiScale(gray, 1.1, 1) #1.1, 1
        cars = np.array([[x, y, x + w, y + h] for (x, y, w, h) in cars])
        pick = non_max_suppression(cars, probs=None, overlapThresh=0.65)
        for (x, y, w, h) in pick:
            # cv2.rectangle(image_scaled, (x, y), (x + w, y + h), (0 , 255, 255), 2) #bgr
            cv2.rectangle(image_scaled, (x, y), (w,  h), (0, 255, 255), 2)
        cv2.imshow('Apasa ESC pentru a iesi din PROGRAM', image_scaled)

            # esc pt iesire
        if cv2.waitKey(33) == 27:
            break
    print('Executie terminata')
    cv2.destroyAllWindows()
    

perform_classification('videos/testvideo_01.avi', 'classifiers/classifier_03.xml')    # autostrada
perform_classification('videos/testvideo_01.avi', 'classifiers/classifier_02.xml')    # autostrada



Executie terminata
Executie terminata


In [2]:
import test_acc as acc
print("Classifier 2 - Utilizat")
acc.get_acc('classifiers/classifier_02.xml')
print("Classifier 1")
acc.get_acc('classifiers/classifier_01.xml')


Classifier 2 - Utilizat
23 autovehicule detectate.
23 autovehicule detectate.
23 autovehicule detectate.
23 autovehicule detectate.
23 autovehicule detectate.
Acuratețe vehicule:  76.66666666666667
Classifier 1
15 autovehicule detectate.
15 autovehicule detectate.
15 autovehicule detectate.
15 autovehicule detectate.
15 autovehicule detectate.
Acuratețe vehicule:  50.0


50.0